In [1]:
# !pip install pythainlp -q
# !pip install --no-cache-dir transformers sentencepiece -q

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

c:\Users\supak\anaconda3\envs\torch2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("../data/bangkok_traffy.csv")
new_df = df.copy()
len(df)

269925

In [4]:
df = df[["type", "comment", "province"]]

In [5]:
# df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]
df = df.drop(["province"], axis=1)

In [6]:
# df['type'].dropna(axis=0, inplace=True)
# df['comment'].dropna(axis=0, inplace=True)

In [7]:
len(df)

269925

In [8]:
df['type'].fillna('', inplace=True) 
df['type'] = df['type'].astype(str) 
df['type'] = df['type'].str.strip('{}').apply(lambda x :[e for e in x.split(',') if e])

In [9]:
# df = df.sample(1000)

In [10]:
df = df.reset_index(drop=True)

In [11]:
df.head()

,type,comment
0,[],ไม่มีภาพ
1,[ความสะอาด],ขยะเยอะ
2,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...


In [12]:
type_columns = ['ป้ายจราจร',
 'ห้องน้ำ',
 'การเดินทาง',
 'คนจรจัด',
 'เสนอแนะ',
 'สอบถาม',
 'PM2.5',
 'สัตว์จรจัด',
 'ป้าย',
 'ร้องเรียน',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']

type_to_idx = {col: index for index, col in enumerate(type_columns)}
idx_to_type = {index: col for index, col in enumerate(type_columns)}

In [13]:
def apply_filter_type(x):
    order_list = ['ป้ายจราจร',
 'ห้องน้ำ',
 'การเดินทาง',
 'คนจรจัด',
 'เสนอแนะ',
 'สอบถาม',
 'PM2.5',
 'สัตว์จรจัด',
 'ป้าย',
 'ร้องเรียน',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']
    for order in order_list:
        if order in x:
            return order
df["type"] = df["type"].apply(apply_filter_type)
df.head()

,type,comment
0,None,ไม่มีภาพ
1,ความสะอาด,ขยะเยอะ
2,สายไฟ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,สะพาน,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,ร้องเรียน,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...


In [14]:
def make_data(x):
  if x != None:
    x = type_to_idx[x]
  else:
    x = -1
  return x
df["type"] = df["type"].apply(make_data)

In [15]:
import re
import string
import pythainlp
from tqdm.auto import tqdm
tqdm.pandas()

def preprocessing_comment(comment):
    if comment == "":
      return commemnt
    # Normalize text
    try:
      comment = pythainlp.util.normalize(comment)
    except:
      return ""
      # print(comment)

    # Tokenize text
    tokens = pythainlp.word_tokenize(comment)

    # Remove stop words
    stopwords = pythainlp.corpus.common.thai_stopwords()
    tokens = [token for token in tokens if token not in stopwords]

    # Join tokens back into a string
    processed_comment = ' '.join(tokens)
  
    return comment
    # return processed_comment

df['comment'] = df['comment'].progress_apply(preprocessing_comment)

100%|██████████| 269925/269925 [06:13<00:00, 721.89it/s] 


In [16]:
df.head()

,type,comment
0,-1,ไม่มีภาพ
1,19,ขยะเยอะ
2,13,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,15,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,9,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...


In [17]:
len(df)

269925

In [18]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [19]:
import torch
from torch.utils.data import Dataset

class MultiLabelTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.feature = self.data["comment"]
        self.labels = self.data["type"].to_numpy()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        comment = self.data['comment'].iloc[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        # Tokenize the comment
        tokenized_comment = self.tokenizer(comment, padding='max_length', truncation=True, max_length=256, return_tensors='pt')

        return {
            'input_ids': tokenized_comment['input_ids'].squeeze(),
            'attention_mask': tokenized_comment['attention_mask'].squeeze(),
            'labels': label
        }


In [20]:
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)
my_dataset = MultiLabelTextDataset(df, tokenizer)
my_loader = DataLoader(my_dataset, batch_size=32, shuffle=False)

In [21]:
# my_dataset[3]

In [22]:
import torch.nn as nn
import torch.optim as optim
import torch

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
class Classifier(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Classifier, self).__init__()
        self.fc2 = nn.Linear(n_inputs, n_outputs)

    def forward(self, x):
        x = self.fc2(x)
        return x

class Model(nn.Module):
    def __init__(self, classifier):
        super().__init__()
        self.embeding = nn.EmbeddingBag(tokenizer.vocab_size+10, 512,padding_idx=1)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)
        self.dropout = nn.Dropout(0.1)
        self.classifier = classifier
        
    def forward(self,input_ids, attention_mask):
        x = self.embeding(input_ids)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        out = self.classifier(x)
        return out

classifier = Classifier(512, len(type_columns))
model = Model(classifier)

In [23]:
# device = torch.device('cpu')
# device

In [24]:
# model.eval()
model.load_state_dict(torch.load('../saved_models/type-classification-model.pth'))
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Model(
  (embeding): EmbeddingBag(25015, 512, mode='mean', padding_idx=1)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Classifier(
    (fc2): Linear(in_features=512, out_features=24, bias=True)
  )
)

In [25]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
criterion = torch.nn.CrossEntropyLoss()
def evaluate_model(data_loader, model, criterion, device):
    valid_correct = 0
    num_valid_samples = 0
    valid_predictions = []
    valid_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            valid_correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

            num_valid_samples += input_ids.size(0)

            valid_predictions.extend(torch.argmax(outputs, dim=1).cpu().tolist())
            valid_labels.extend(labels.cpu().tolist())

    valid_accuracy = valid_correct / num_valid_samples
    valid_precision = precision_score(valid_labels, valid_predictions, average='micro')
    valid_recall = recall_score(valid_labels, valid_predictions, average='micro')
    valid_f1 = f1_score(valid_labels, valid_predictions, average='micro')

    return valid_accuracy, valid_precision, valid_recall, valid_f1, valid_predictions

tmp = evaluate_model(my_loader, model, criterion, device)
valid_predictions = tmp[-1]

100%|██████████| 8436/8436 [05:48<00:00, 24.19it/s]


In [26]:
# valid_predictions

In [28]:
tmp_df = df.copy()
predictions = valid_predictions
def update_value(row, index, predictions):
    if row == -1:
        return predictions[index]
    return row

# Iterate over the DataFrame rows and update values
for i, row in tqdm(tmp_df.iterrows()):
    tmp_df.loc[i, 'type'] = update_value(row['type'], i, predictions)


269925it [01:20, 3342.95it/s]


In [29]:
df

,type,comment
0,-1,ไม่มีภาพ
1,19,ขยะเยอะ
2,13,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,15,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,9,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
...,...,...
269920,-1,
269921,13,สายไฟระโยงระยางมากค่ะ ช่วยมาจัดการเรื่องสายไฟด...
269922,23,เทศกิจ ติดป่ายจะย้าย รถ\nปัจจุบัน รถเก่าของอู่...
269923,22,ฟุตบาทชำรุด-สกปรก


In [30]:
tmp_df

,type,comment
0,21,ไม่มีภาพ
1,19,ขยะเยอะ
2,13,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,15,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,9,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
...,...,...
269920,11,
269921,13,สายไฟระโยงระยางมากค่ะ ช่วยมาจัดการเรื่องสายไฟด...
269922,23,เทศกิจ ติดป่ายจะย้าย รถ\nปัจจุบัน รถเก่าของอู่...
269923,22,ฟุตบาทชำรุด-สกปรก


In [31]:
tmp_df["type"] = tmp_df["type"].apply(lambda x: idx_to_type[x])
tmp_df

,type,comment
0,แสงสว่าง,ไม่มีภาพ
1,ความสะอาด,ขยะเยอะ
2,สายไฟ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,สะพาน,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,ร้องเรียน,น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
...,...,...
269920,คลอง,
269921,สายไฟ,สายไฟระโยงระยางมากค่ะ ช่วยมาจัดการเรื่องสายไฟด...
269922,ถนน,เทศกิจ ติดป่ายจะย้าย รถ\nปัจจุบัน รถเก่าของอู่...
269923,ทางเท้า,ฟุตบาทชำรุด-สกปรก


In [32]:
len(tmp_df)

269925

In [40]:
predicted_type = tmp_df.type.tolist()

In [43]:
new_df_impute = new_df.copy()
def update_value_2(row, index, predictions):
    if row == "{}":
        return "{"+str(predictions[index])+"}"
    return row

# Iterate over the DataFrame rows and update values
for i, row in tqdm(new_df_impute.iterrows()):
    new_df_impute.loc[i, 'type'] = update_value_2(row['type'], i, predicted_type)
new_df_impute

269925it [10:27, 430.04it/s]


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-9LHDM6,{แสงสว่าง},NaN,ไม่มีภาพ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.48661,13.79386",1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุง...,บางพลัด,บางพลัด,กรุงเทพมหานคร,2021-09-01 10:44:55.353209+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:59:58.622268+00
1,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,NaN,2022-06-04 15:34:14.609206+00
2,2021-8GKAR9,{สายไฟ},ยังไม่มีหน่วยงานรับผิดชอบ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.57685,13.79704",335/31 ลาดพร้าว แขวง สามเสนนอก เขตห้วยขวาง กรุ...,สามเสนนอก,ห้วยขวาง,จังหวัดกรุงเทพมหานคร,2021-09-19 06:47:50.488685+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:07.293416+00
3,2021-AFPUXZ,"{ถนน,สะพาน}",NaN,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52916,13.72338",37 10 ซอย สีลม 9 แขวง สีลม เขตบางรัก กรุงเทพมห...,สีลม,บางรัก,กรุงเทพมหานคร,2021-09-19 07:40:30.000781+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:04.507406+00
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,NaN,2022-06-21 08:21:09.532782+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269920,NaN,{คลอง},กรุงเทพโปร่งใส,NaN,https://storage.googleapis.com/traffy_public_b...,NaN,"100.55580,13.72502",NaN,คลองเตย,คลองเตย,กรุงเทพมหานคร,2023-05-11 02:50:30.856774+00,รอรับเรื่อง,NaN,0.0,2023-05-11 02:50:30.847824+00
269921,2023-AY9WFK,{สายไฟ},เขตลาดกระบัง,สายไฟระโยงระยางมากค่ะ ช่วยมาจัดการเรื่องสายไฟด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.74688,13.74099",2 1 ซอย ร่มเกล้า 25/2 แขวงคลองสามประเวศ เขตลาด...,คลองสามประเวศ,ลาดกระบัง,กรุงเทพมหานคร,2023-05-11 02:50:47.768422+00,รอรับเรื่อง,NaN,0.0,2023-05-11 02:50:47.75879+00
269922,2023-GPURT9,{ถนน},เขตภาษีเจริญ,เทศกิจ ติดป่ายจะย้าย รถ\n\nปัจจุบัน รถเก่าของอ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.44695,13.73987",49 ถ. บางแวก แขวงบางแวก เขตภาษีเจริญ กรุงเทพมห...,บางแวก,ภาษีเจริญ,กรุงเทพมหานคร,2023-05-11 02:51:44.186578+00,รอรับเรื่อง,NaN,0.0,2023-05-11 02:51:44.173161+00
269923,2023-6BW3WV,{ทางเท้า},เขตจตุจักร,ฟุตบาทชำรุด-สกปรก,https://storage.googleapis.com/traffy_public_b...,NaN,"100.57728,13.80415",1111/5 ถ. ลาดพร้าว แขวงจันทรเกษม เขตจตุจักร กร...,จันทรเกษม,จตุจักร,กรุงเทพมหานคร,2023-05-11 02:52:28.347238+00,รอรับเรื่อง,NaN,0.0,2023-05-11 02:52:28.337167+00


In [44]:
new_df_impute.to_csv("../data/imputed_type_bangkok_traffy.csv")